# Part 2: 모델 임포트 및 Sequential 모델 학습

이 노트북은 Part 1에서 이어집니다.

## 5. 모델 임포트

In [1]:
# 모델 임포트
import sys
sys.path.append('models')

from bam_sequential import SequentialBAM
from bam_mtl import MTLBAM
from cae_sequential import SequentialCAE
from cae_mtl import CAEMTL
from unet_sequential import SequentialUNet
from unet_mtl import UNetMTL

print("✓ All models imported successfully")

2025-10-27 01:45:54.137176: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-27 01:45:54.191535: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-10-27 01:45:54.191561: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-10-27 01:45:54.192239: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-10-27 01:45:54.196857: I tensorflow/core/platform/cpu_feature_guar

✓ All models imported successfully


## 6. 학습 시작 - Sequential Models

### 6.1 Sequential BAM

In [2]:
print("\n" + "="*80)
print("Model 1/6: Sequential BAM")
print("="*80)

# 모델 생성
seq_bam = SequentialBAM(
    input_dim=3072,  # 32*32*3
    denoise_latent=256,
    cls_latent=128,
    num_classes=10
)

# 모델 컴파일
seq_bam.compile_models(
    denoise_lr=INITIAL_LR,
    cls_lr=INITIAL_LR
)

print("\n[Stage 1: Denoising BAM]")
seq_bam.denoise_model.summary()

# Stage 1 학습
callbacks_stage1 = get_callbacks('sequential_bam_stage1', monitor='val_mse', patience=30)
history_stage1 = seq_bam.train_stage1(
    x_train_flat,
    x_train_clean_flat,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_split=VALIDATION_SPLIT,
    callbacks=callbacks_stage1
)

# Stage 1 결과 저장
save_history(history_stage1, 'sequential_bam_stage1')
seq_bam.denoise_model.save('weights/sequential_bam_denoise.keras')
print("✓ Stage 1 model saved")

print("\n[Stage 2: Classification BAM]")
seq_bam.cls_model.summary()

# Stage 2 학습
callbacks_stage2 = get_callbacks('sequential_bam_stage2', monitor='val_acc', patience=30)
history_stage2 = seq_bam.train_stage2(
    x_train_flat,
    y_train_augmented,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_split=VALIDATION_SPLIT,
    callbacks=callbacks_stage2
)

# Stage 2 결과 저장
save_history(history_stage2, 'sequential_bam_stage2')
seq_bam.cls_model.save('weights/sequential_bam_classification.keras')
print("✓ Stage 2 model saved")

# 평가
print("\n[Evaluation]")
results = seq_bam.evaluate(x_test_flat, x_test_clean_flat, y_test_augmented, batch_size=BATCH_SIZE)
save_results(results, 'sequential_bam')

print("\nSequential BAM Results:")
print(f"  Restoration MSE: {results['restoration']['mse']:.6f}")
print(f"  Restoration PSNR: {results['restoration']['psnr']:.2f} dB")
print(f"  Classification Accuracy: {results['classification']['accuracy']:.4f}")

# 메모리 정리
del seq_bam
clear_memory()


Model 1/6: Sequential BAM


2025-10-27 01:45:56.001195: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-10-27 01:45:56.036294: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-10-27 01:45:56.036338: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-10-27 01:45:56.039910: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-10-27 01:45:56.039996: I external/local_xla/xla/stream_executor

NameError: name 'INITIAL_LR' is not defined

### 6.2 Sequential CAE

In [ ]:
print("\n" + "="*80)
print("Model 2/6: Sequential CAE")
print("="*80)

# 모델 생성
seq_cae = SequentialCAE(
    input_shape=(32, 32, 3),
    num_classes=10
)

# 모델 컴파일
seq_cae.compile_models(
    restore_lr=INITIAL_LR,
    cls_lr=INITIAL_LR,
    restore_loss='mae'
)

print("\n[Stage 1: Restoration CAE]")
seq_cae.restore_model.summary()

# Stage 1 학습
callbacks_stage1 = get_callbacks('sequential_cae_stage1', monitor='val_mae', patience=30)
history_stage1 = seq_cae.train_stage1(
    x_train_augmented,
    np.repeat(x_train_clean, 3, axis=0),  # 원본 데이터 3배 복제
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_split=VALIDATION_SPLIT,
    callbacks=callbacks_stage1
)

# Stage 1 결과 저장
save_history(history_stage1, 'sequential_cae_stage1')
seq_cae.restore_model.save('weights/sequential_cae_restore.keras')
print("✓ Stage 1 model saved")

print("\n[Stage 2: Classification CAE]")
seq_cae.cls_model.summary()

# Stage 2 학습
callbacks_stage2 = get_callbacks('sequential_cae_stage2', monitor='val_accuracy', patience=30)
history_stage2 = seq_cae.train_stage2(
    x_train_augmented,
    y_train_augmented,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_split=VALIDATION_SPLIT,
    callbacks=callbacks_stage2
)

# Stage 2 결과 저장
save_history(history_stage2, 'sequential_cae_stage2')
seq_cae.cls_model.save('weights/sequential_cae_classification.keras')
print("✓ Stage 2 model saved")

# 평가
print("\n[Evaluation]")
results = seq_cae.evaluate(
    x_test_augmented,
    np.repeat(x_test_clean, 3, axis=0),
    y_test_augmented,
    batch_size=BATCH_SIZE
)
save_results(results, 'sequential_cae')

print("\nSequential CAE Results:")
print(f"  Restoration MSE: {results['restoration']['mse']:.6f}")
print(f"  Restoration PSNR: {results['restoration']['psnr']:.2f} dB")
print(f"  Classification Accuracy: {results['classification']['accuracy']:.4f}")

# 메모리 정리
del seq_cae
clear_memory()

### 6.3 Sequential U-Net

In [ ]:
print("\n" + "="*80)
print("Model 3/6: Sequential U-Net")
print("="*80)

# 모델 생성
seq_unet = SequentialUNet(
    input_shape=(32, 32, 3),
    num_classes=10,
    restore_dropout=0.0,
    cls_dropout=0.1
)

# 모델 컴파일
seq_unet.compile_models(
    restore_lr=INITIAL_LR,
    cls_lr=INITIAL_LR,
    restore_loss='mae'
)

print("\n[Stage 1: Restoration U-Net]")
seq_unet.restore_model.summary()

# Stage 1 학습
callbacks_stage1 = get_callbacks('sequential_unet_stage1', monitor='val_mae', patience=30)
history_stage1 = seq_unet.train_stage1(
    x_train_augmented,
    np.repeat(x_train_clean, 3, axis=0),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_split=VALIDATION_SPLIT,
    callbacks=callbacks_stage1
)

# Stage 1 결과 저장
save_history(history_stage1, 'sequential_unet_stage1')
seq_unet.restore_model.save('weights/sequential_unet_restore.keras')
print("✓ Stage 1 model saved")

print("\n[Stage 2: Classification U-Net]")
seq_unet.cls_model.summary()

# Stage 2 학습
callbacks_stage2 = get_callbacks('sequential_unet_stage2', monitor='val_accuracy', patience=30)
history_stage2 = seq_unet.train_stage2(
    x_train_augmented,
    y_train_augmented,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_split=VALIDATION_SPLIT,
    callbacks=callbacks_stage2
)

# Stage 2 결과 저장
save_history(history_stage2, 'sequential_unet_stage2')
seq_unet.cls_model.save('weights/sequential_unet_classification.keras')
print("✓ Stage 2 model saved")

# 평가
print("\n[Evaluation]")
results = seq_unet.evaluate(
    x_test_augmented,
    np.repeat(x_test_clean, 3, axis=0),
    y_test_augmented,
    batch_size=BATCH_SIZE
)
save_results(results, 'sequential_unet')

print("\nSequential U-Net Results:")
print(f"  Restoration MSE: {results['restoration']['mse']:.6f}")
print(f"  Restoration PSNR: {results['restoration']['psnr']:.2f} dB")
print(f"  Classification Accuracy: {results['classification']['accuracy']:.4f}")

# 메모리 정리
del seq_unet
clear_memory()

## 7. Sequential 모델 학습 완료

3개의 Sequential 모델 학습이 완료되었습니다.

다음 Part 3에서 MTL 모델들을 학습하겠습니다.